In [1]:
# Install Pytorch & other libraries
%pip install "torch==2.4.0" tensorboard pillow

# Install Hugging Face libraries
%pip install  --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils" 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 16.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 32.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 45.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 34.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 28.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [1]:

import torch

# GPU가 Flash Attention을 지원하는지 확인
assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'

# Flash Attention 설치
!pip install ninja packaging
!pip install flash-attn --no-build-isolation --upgrade


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# import streamlit as st
import pandas as pd
# import fitz
import io
import time
import json
import os
import urllib.request
from datetime import datetime, timedelta
import requests
import pickle

In [4]:
%cd /workspace/stk-analyst-chatbot

/workspace/stk-analyst-chatbot


In [5]:
with open('data/multiturn_functioncall.pkl' ,'rb' )as f:
    final_train_messages =  pickle.load(f)

In [6]:
messeges = [{'message':d} for d in final_train_messages]
from datasets import Dataset

dataset = Dataset.from_list(messeges)

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2.5-7B-Instruct" 

# BitsAndBytes 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,                             # 4비트 양자화 사용
   bnb_4bit_use_double_quant=True,               # 이중 양자화 사용으로 메모리 추가 절약
   bnb_4bit_quant_type="nf4",                    # 4비트 양자화 타입 설정(normalized float 4)
   bnb_4bit_compute_dtype=torch.bfloat16         # 연산 시 bfloat16 타입 사용
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    dataset[0]["message"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하며, 각 함수의 사용 방법과 파라미터는 <tools></tools> XML 태그 안에 명시되어 있습니다. 함수를 호출할 때는 사용자가 제공한 값에 따라 정확히 실행해야 합니다.

사용 가능한 함수와 각 파라미터의 설명은 다음과 같습니다:

1. get_issues_summarized
   - 설명: 특정 회사나 키워드에 대한 이슈를 검색하고 요약합니다.
   - 파라미터:
     - keyword: 검색하고자 하는 회사명 또는 키워드.
     - days: 검색 기간(일 단위).

2. get_reddit_hotissue
   - 설명: 금융 시장에서 현재 주목받고 있는 이슈를 요약합니다.
   - 파라미터:
     - days: 검색 기간(일 단위).

3. get_earnings
   - 설명: 기업의 재무재표나 현금흐름을 가져오고 분석합니다. 이전 년도 데이터가 필요한 경우, 해당 데이터를 추가로 호출합니다.
           함수 호출 시 연도와 분기 설정에 주의해야 합니다. 보통 1분기 실적발표는 4월 중순 ~ 5월 초, 2분기는 7월 중순 ~ 8월초, 3분기는 10월중순~ 11월 초, 
            4분기는 1월 중순 ~2월초에 발표 된다. 
            ## 예시: 1. 오늘 날짜 2025년 2월 18일 , 최근 실적발표의 year  과 quarter 은?
            year : '2024' , quarter: -1 또는, 4 
   - 파라미터:
     - symbol: 데이터를 검색하고자 하는 기업 심볼.
     - analysis_type: 분석 유형(growth, profitability, stability, valuation, cashflow, dividend, cost, NA).
     - type_: 데이터 타입(yearly 또는 quarter).
     - year

In [9]:
from peft import LoraConfig
 
# QLoRA 논문 및 Sebastian Raschka 실험에 기반한 LoRA Conifg
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0,
        r=16,
        bias="none",
        target_modules=["q_proj", "k_proj"],
        task_type="CAUSAL_LM",
)


# from transformers import TrainingArguments
from trl import SFTConfig

args = SFTConfig(
    output_dir="qwen2.5-14b-functioncall",     # 저장될 디렉토리와 저장소 ID
    num_train_epochs=10,                      # 학습할 총 에포크 수
    per_device_train_batch_size=4,           # 장치당 학습 배치 크기
    gradient_accumulation_steps=2,           # 역전파/가중치 업데이트 전 누적할 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 그래디언트 체크포인팅 사용
    optim="adamw_torch_fused",               # 퓨즈드 AdamW 옵티마이저 사용
    logging_steps=10,                        # 10스텝마다 로그 기록
    save_strategy="steps",                   # 특정 스텝마다 체크포인트 저장
    save_steps=50,
    bf16=True,                              # bfloat16 정밀도 사용
    tf32=True,                              # tf32 정밀도 사용
    learning_rate=1e-4,                     # 학습률 (QLoRA 논문 기반)
    max_grad_norm=0.3,                      # 최대 그래디언트 노름 (QLoRA 논문 기반)
    warmup_ratio=0.03,                      # 워밍업 비율 (QLoRA 논문 기반)
    lr_scheduler_type="constant",           # 고정 학습률 스케줄러 사용
    push_to_hub=False,                      # 허브에 모델 업로드 안 함
    report_to="tensorboard",                # 텐서보드에 메트릭 기록
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True}
)


In [10]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["message"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

def print_tokens_and_labels(batch):
    input_ids = batch["input_ids"][0].tolist()
    labels = batch["labels"][0].tolist()
    
    print("\n토큰과 레이블 비교:")
    print(f"{'Token ID':<10} {'Token':<30} {'Label':<10}")
    print("-" * 50)
    
    for token_id, label in zip(input_ids, labels):
        token = tokenizer.decode([token_id])
        label_str = str(label) if label != -100 else "-100"
        print(f"{token_id:<10} {token:<30} {label_str:<10}")

In [11]:
example = dataset[0]

In [12]:
# collate_fn 테스트 (배치 크기 1로)
max_seq_length = 8192  
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1818])
어텐션 마스크 형태: torch.Size([1, 1818])
레이블 형태: torch.Size([1, 1818])


In [13]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

입력에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [14]:
from trl import SFTTrainer

max_seq_length = 8192  # 모델과 데이터셋 패킹을 위한 최대 시퀀스 길이

trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [ ]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_aut

Step,Training Loss
10,0.951000
20,0.855700
30,0.863700
40,0.751000
50,0.736900
60,0.758900
70,0.732800
80,0.729100
90,0.682400
100,0.668000


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [ ]:
train.save()